This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "voc-10609208041266773387460669f72809407a3.29559041"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.llms import OpenAI


In [2]:
!pip install chromadb==0.4.12

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import openai
import os

import pandas as pd

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain


In [4]:
MODEL_NAME = "gpt-4o"
API_KEY = "voc-10609208041266773387460669f72809407a3.29559041"

model = OpenAI(model_name=MODEL_NAME, temperature=0, api_key=API_KEY)

prompt="""
Create 10 real estate listings given the example below. Seperate each listing with a "!!!"
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
"""

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [5]:
'''
listings = model(prompt)
listings
listings = listings.split("!!!")
df = pd.DataFrame(listings, columns=['Listings'])
df.to_csv('homes.csv', index=False)
'''

'\nlistings = model(prompt)\nlistings\nlistings = listings.split("!!!")\ndf = pd.DataFrame(listings, columns=[\'Listings\'])\ndf.to_csv(\'homes.csv\', index=False)\n'

In [6]:
df_loaded = pd.read_csv('listings.csv')
print(df_loaded)

                                             Listings
0   Neighborhood: Maplewood Heights\nPrice: $950,0...
1   Neighborhood: Sunnyvale\nPrice: $720,000\nBedr...
2   Neighborhood: Riverwood Estates\nPrice: $1,200...
3   Neighborhood: Willow Creek\nPrice: $650,000\nB...
4   Neighborhood: Pine Ridge\nPrice: $780,000\nBed...
5   Neighborhood: Cedar Grove\nPrice: $890,000\nBe...
6   Neighborhood: Oakwood Hills\nPrice: $1,050,000...
7   Neighborhood: Birchwood Park\nPrice: $700,000\...
8   Neighborhood: Elmwood Village\nPrice: $820,000...
9   Neighborhood: Redwood Meadows\nPrice: $980,000...
10  Neighborhood: Aspen Ridge\nPrice: $1,150,000\n...


In [7]:
loader = CSVLoader(file_path='./homes.csv')
docs = loader.load()

In [8]:
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

In [9]:
embeddings = OpenAIEmbeddings()

db = Chroma.from_documents(split_docs, embeddings)

In [17]:
questions = [   
                "How big do you want your house to be?",
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
answers = [
    "I want a small house for me and my dog",
    "A nice view, good local food, and only one floor.",
    "A backyard for my dog, a parking spot, and a large enough kitchen area for friends and family.",
    "Walkable areas, access to at least one highway",
    "Urban with a enough trees to not feel away from nature"]

In [18]:
query = f"""
    {questions[0]} {answers[0]} 
    {questions[1]} {answers[1]}
    {questions[2]} {answers[2]}
    {questions[3]} {answers[3]}
    {questions[4]} {answers[4]}
    
    Use these question and answers to provide the best listing. Create a small sumamry after the listing that emphasizes their wants and needs.
    """

use_chain_helper = False
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
    print(rag.run(query))
else:
    similar_docs = db.similarity_search(query, k=5)
    prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )
    chain = load_qa_chain(model, prompt = prompt, chain_type="stuff")
    print(chain.run(input_documents=similar_docs, query = query))

### Best Listing for Your Needs

**Neighborhood: Willow Creek**
- **Price:** $650,000
- **Bedrooms:** 3
- **Bathrooms:** 2.5
- **House Size:** 2,100 sqft

**Description:**
Welcome to this charming 3-bedroom, 2.5-bathroom home in Willow Creek. The open-concept living area features a cozy fireplace and hardwood floors. The kitchen is a chef's dream with granite countertops, stainless steel appliances, and a breakfast nook. The master suite includes a walk-in closet and a luxurious bathroom with a soaking tub. The backyard is perfect for entertaining with a large deck and mature trees providing shade and privacy.

**Neighborhood Description:**
Willow Creek is a picturesque community with tree-lined streets, parks, and excellent schools. Enjoy the local shops, cafes, and restaurants, as well as easy access to hiking and biking trails. Willow Creek offers a perfect blend of suburban tranquility and urban convenience.

### Summary

This Willow Creek property is an excellent match for your ne

In [ ]:
'''
questions = [   
                "How big do you want your house to be?",
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."]
    
HomeMatch:
Based on your preferences, the best listing for you would be the property in **Birchwood Park**. Here’s why:

### Listing: Birchwood Park
**Price:** $700,000  
**Bedrooms:** 3  
**Bathrooms:** 2  
**House Size:** 2,000 sqft  

**Description:**  
Welcome to this charming 3-bedroom, 2-bathroom home in Birchwood Park. The open-concept living area features a cozy fireplace and hardwood floors. The kitchen is a chef's dream with granite countertops, stainless steel appliances, and a breakfast nook. The master suite includes a walk-in closet and a luxurious bathroom with a soaking tub. The backyard is perfect for entertaining with a large deck and mature trees providing shade and privacy.

**Neighborhood Description:**  
Birchwood Park is a picturesque community with tree-lined streets, parks, and excellent schools. Enjoy the local shops, cafes, and restaurants, as well as easy access to hiking and biking trails. Birchwood Park offers a perfect blend of suburban tranquility and urban convenience.

### Why Birchwood Park is the Best Fit:

1. **House Size and Layout:**
   - The 3-bedroom, 2-bathroom layout matches your requirement for a comfortable three-bedroom house.
   - The open-concept living area and spacious kitchen align with your desire for a cozy living room and a spacious kitchen.

2. **Neighborhood:**
   - Birchwood Park offers a quiet, picturesque community, which aligns with your preference for a quiet neighborhood.
   - The excellent schools in the area meet your requirement for good local schools.

3. **Amenities:**
   - The backyard with a large deck is perfect for gardening and outdoor activities.
   - Although it doesn't explicitly mention a two-car garage, the overall property size and price suggest that it could be a feature worth inquiring about.
   - The modern kitchen and luxurious bathroom indicate a focus on quality and comfort, which may include energy-efficient systems.

4. **Transportation:**
   - Birchwood Park offers easy access to hiking and biking trails, which aligns with your preference for bike-friendly roads.
   - While the listing does not explicitly mention proximity to a major highway or a reliable bus line, the urban convenience aspect suggests good connectivity.

5. **Urban Balance:**
   - Birchwood Park provides a perfect blend of suburban tranquility and urban convenience, matching your desire for a balance between suburban and urban amenities.

### Additional Considerations:
- **Price:** At $700,000, this property is within your budget and offers good value for the features and amenities provided.
- **House Size:** With 2,000 sqft, this home offers ample space for comfortable living.

Overall, Birchwood Park seems to be the best match for your needs and preferences.'''

In [ ]:
'''### Best Listing for Your Needs

**Neighborhood: Willow Creek**
- **Price:** $650,000
- **Bedrooms:** 3
- **Bathrooms:** 2.5
- **House Size:** 2,100 sqft

**Description:**
Welcome to this charming 3-bedroom, 2.5-bathroom home in Willow Creek. The open-concept living area features a cozy fireplace and hardwood floors. The kitchen is a chef's dream with granite countertops, stainless steel appliances, and a breakfast nook. The master suite includes a walk-in closet and a luxurious bathroom with a soaking tub. The backyard is perfect for entertaining with a large deck and mature trees providing shade and privacy.

**Neighborhood Description:**
Willow Creek is a picturesque community with tree-lined streets, parks, and excellent schools. Enjoy the local shops, cafes, and restaurants, as well as easy access to hiking and biking trails. Willow Creek offers a perfect blend of suburban tranquility and urban convenience.

### Summary

This Willow Creek property is an excellent match for your needs and preferences. The single-story layout ensures ease of access, and the spacious backyard is perfect for your dog to roam freely. The open-concept living area and large kitchen provide ample space for entertaining friends and family. The neighborhood offers a blend of urban convenience and natural beauty, with tree-lined streets and easy access to local shops, cafes, and restaurants. Additionally, the proximity to hiking and biking trails ensures you and your dog can enjoy the outdoors. The price is also within your budget, making this home a great fit for you.'''